In [1]:
from keras.datasets import fashion_mnist

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, InputLayer, BatchNormalization
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.callbacks import TensorBoard
import keras

Using TensorFlow backend.


## training params

In [2]:

batch_size = 128
epochs = 5

pool_size = (2, 2)                    # size of poolin area for max pooling
prob_drop_conv = 0.25                  # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5                # drop probability for dropout @ fc layer

## data params

In [3]:
# Create dictionary of target classes
label_dict = {
 0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot'
}

# data params
nb_classes = len(label_dict)

## Load MNIST dataset

In [4]:
(X_train_raw, Y_train_raw), (X_test_raw, Y_test_raw) = fashion_mnist.load_data()
print('X_train_raw original shape:', X_train_raw.shape)

X_train_raw original shape: (60000, 28, 28)


### Reshape Data from image matrix to array & normalize


In [5]:
if K.image_dim_ordering() == 'th':

    # Theano
    X_train = X_train_raw.reshape(X_train_raw.shape[0], 1, X_train_raw.shape[1], X_train_raw.shape[2]).astype('float32')
    X_train /= 255.

    X_test = X_test_raw.reshape(X_test_raw.shape[0], 1, X_test_raw.shape[1], X_test_raw.shape[2]).astype('float32')
    X_test /= 255.
    
    input_shape = (1, X_train_raw.shape[1], X_train_raw.shape[2])
    
else:
    
    # TensorFlow
    X_train = X_train_raw.reshape(X_train_raw.shape[0], X_train_raw.shape[1], X_train_raw.shape[2], 1).astype('float32')
    X_train /= 255.

    X_test = X_test_raw.reshape(X_test_raw.shape[0], X_test_raw.shape[1], X_test_raw.shape[2], 1).astype('float32')
    X_test /= 255.
    
    input_shape = (X_train_raw.shape[1], X_train_raw.shape[2], 1)
    
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('input_shape: ', input_shape)

X_train shape:  (60000, 28, 28, 1)
X_test shape:  (10000, 28, 28, 1)
input_shape:  (28, 28, 1)


In [6]:
Y_train = np_utils.to_categorical(Y_train_raw)

Y_test = np_utils.to_categorical(Y_test_raw)

## Convolutional Network model

In [7]:
model = Sequential()

# input layer
model.add(InputLayer(input_shape=input_shape))

# batch normalization
model.add(BatchNormalization())

# conv1 layer
model.add(Conv2D(32, (3,3), padding='same', kernel_initializer='random_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2, 2), padding='same'))
model.add(Dropout(prob_drop_conv))

# conv2 layer
model.add(Conv2D(64, (3,3), padding='same', kernel_initializer='random_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2, 2), padding='same'))
model.add(Dropout(prob_drop_conv))

# conv3 layer
model.add(Conv2D(128, (3,3), padding='same', kernel_initializer='random_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2, 2), padding='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
# why is it 625 here??
model.add(Dense(units=250, kernel_initializer='random_normal'))
model.add(Activation('relu'))
model.add(Dropout(prob_drop_hidden))

# fc2 layer
model.add(Dense(units=10, kernel_initializer='random_normal'))
model.add(Activation('softmax'))
          
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
__________

## Train

In [ ]:
tensorboard_callback = TensorBoard(log_dir='./logs/mnist_tensorboard', histogram_freq=1, write_graph=True, write_grads=True, batch_size=2*batch_size, write_images=True)

In [ ]:
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_split=0.15, verbose=1, callbacks=[tensorboard_callback])

Train on 51000 samples, validate on 9000 samples
Epoch 1/20
51000/51000 [==============================] - 112s 2ms/step - loss: 0.6882 - acc: 0.7452 - val_loss: 0.3962 - val_acc: 0.8508
Epoch 2/20
51000/51000 [==============================] - 112s 2ms/step - loss: 0.4218 - acc: 0.8473 - val_loss: 0.3239 - val_acc: 0.8814
Epoch 3/20
51000/51000 [==============================] - 111s 2ms/step - loss: 0.3605 - acc: 0.8671 - val_loss: 0.2906 - val_acc: 0.8912
Epoch 4/20
51000/51000 [==============================] - 111s 2ms/step - loss: 0.3286 - acc: 0.8799 - val_loss: 0.2791 - val_acc: 0.8968
Epoch 5/20
51000/51000 [==============================] - 110s 2ms/step - loss: 0.3035 - acc: 0.8887 - val_loss: 0.2467 - val_acc: 0.9058
Epoch 6/20
42752/51000 [========================>.....] - ETA: 24s - loss: 0.2356 - acc: 0.9132

In [ ]:
history.history

## Evaluate

In [ ]:
evaluation = model.evaluate(X_test, Y_test, batch_size=2*batch_size, verbose=1)
print('Summary: Loss over the test dataset: %.4f, Accuracy: %.4f' % (evaluation[0], evaluation[1]))

## Predict

In [ ]:
test_predictions = model.predict(X_test)
test_prediction_ids = test_predictions.argmax(axis=1)

## Plot some predictions

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
def plots(ims, figsize=(12,1), dpi=150, rows=1, interp=False, titles=None):
    f = plt.figure(figsize=figsize, dpi=dpi)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        if titles is not None:
            sp.set_title(titles[i], fontsize=8)
        plt.imshow(ims[i], interpolation=None if interp else 'none', cmap='Greys')

In [ ]:
n_view = 10

idx = np.random.permutation(range(0,len(X_test)))[:n_view]

imgs = X_test[idx,:].reshape(n_view, X_train_raw.shape[1], X_train_raw.shape[2])
titles = [label_dict.get(key) for key in test_prediction_ids[idx]]

In [ ]:
plots(imgs, titles=titles)
plt.show()

## Examine wrong predictions


In [ ]:
# convert category vector into category label
Y_test_ids = Y_test.argmax(axis=1)

# compare the predicted and actual category labels to find error predictions
errors = np.array(Y_test_ids) - np.array(test_prediction_ids)

# gather all the wrong prediction indices
errors_idx = np.nonzero(errors)[0]

In [ ]:
n_view = 10

random_idx = np.random.permutation(range(0,len(errors_idx)))[:n_view]

# grab a sampling of error indices 
random_errors_idx = errors_idx[random_idx]

# and prepare their images and labels
imgs = X_test[random_errors_idx,:].reshape(n_view, X_test_raw.shape[1], X_test_raw.shape[2])

# convert category_id to label string
prediction_titles = [label_dict.get(category_id) for category_id in test_prediction_ids[random_errors_idx]]
correct_titles = [label_dict.get(category_id) for category_id in Y_test_ids[random_errors_idx]]

# construct title string as "predicted>actual"
title_tuples = zip(prediction_titles, correct_titles)
titles = ['>'.join(t) for t in title_tuples]

In [ ]:
plots(imgs, titles=titles)
plt.show()

In [ ]:
len(errors_idx) / 10000.
